In [4]:
import pandas as pd
import numpy as np
import cirpy


In [5]:
compound_fp = './database/Compound.csv'


content_fp = './database/Content.csv'
drug_info_comb_fp = './data/Drug_info_combined.csv'
food_fp = './database/Food.csv'

pd.set_option('display.max_columns', None)

In [64]:
compound_df = pd.read_csv(compound_fp)
# for i in compound_df:
#     display(i)
#     break
compound_df.head(2)

,id,public_id,name,moldb_iupac,state,annotation_quality,description,cas_number,moldb_inchikey,moldb_inchi,moldb_smiles,moldb_mono_mass,kingdom,superklass,klass,subklass
0,4,FDB000004,Cyanidin 3-(6''-acetyl-galactoside),NaN,low,Constituent of the leaves of Nymphaea alba [CC...,350602-26-5,[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,InChI=1S/C23H22O12/c1-9(24)32-8-18-19(29)20(30...,491.118951,HBXXDBKJLPLXPR-DLBZZEGUSA-O,"3-{[(2S,3R,4S,5R,6R)-6-[(acetyloxy)methyl]-3,4...",Organic compounds,Phenylpropanoids and polyketides,Flavonoids,Flavonoid glycosides
1,13,FDB000013,Cyanidin 3-(6''-succinyl-glucoside),NaN,low,Constituent of Phragmites australis [CCD]. Cya...,216692-08-9,[H][C@]1(COC(=O)CCC(O)=O)O[C@@]([H])(OC2=CC3=C...,InChI=1S/C25H24O14/c26-11-6-14(28)12-8-17(24(3...,549.124431,MIYGQTFETYBMKF-WVXUANQFSA-O,"7-methoxy-2,2-dimethyl-2H-chromene-6-carboxyli...",Organic compounds,Phenylpropanoids and polyketides,Flavonoids,Flavonoid glycosides


In [98]:
food_compounds = pd.DataFrame([])
content_df = pd.read_csv(content_fp, chunksize = 200_000)

cnt = 0
for chunk in content_df:
#     cnt += 1
    foo = chunk.loc[chunk.source_type == 'Compound', ['source_id',
                                                    'food_id', 
                                                    'orig_food_common_name',
                                                    'orig_content',
                                                    'orig_unit']]
    foo = foo.drop_duplicates().dropna(subset=['orig_food_common_name'])
#     display(foo)
    
    foo['orig_food_common_name'] = foo\
        .orig_food_common_name.apply(lambda x: x.split(',')[0].lower().strip())
    foo = foo\
            .groupby('orig_food_common_name')\
            .apply(lambda df: df.nlargest(5, 'orig_content'))
#     display(foo)
    food_compounds = pd.concat([food_compounds, foo], axis=0)
#     if cnt == 1:
#         break
# compound_df['']


/Applications/anaconda3/envs/deepddi2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Applications/anaconda3/envs/deepddi2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,5,6,7,8,9,13,14,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Applications/anaconda3/envs/deepddi2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,6,7,8,9,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Applications/anaconda3/envs/deepddi2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,7,9,13,21,22) have m

In [121]:
compound_df_proc = compound_df[['id', 'name', 'cas_number']]\
                            .dropna(subset = ['cas_number'])\
                            .drop_duplicates()
combo = food_compounds.merge(compound_df_proc, left_on = 'source_id', right_on = 'id').drop_duplicates()
final_combo = combo.groupby(['orig_food_common_name', 'name'])\
        .apply(lambda df: pd.DataFrame({'orig_content':[np.mean(df.orig_content)],
                                        'orig_unit': df.orig_unit.values[0],
                                        'cas_number': df.cas_number.values[0]}))\
        .reset_index()

In [137]:
final_combo_fp = './database/food_compounds_lookup.csv'
final_combo.drop(columns = ['level_2']).to_csv(final_combo_fp, index = None)


In [136]:
final_combo#.groupby(['orig_food_common_name']).count().max()

,orig_food_common_name,name,level_2,orig_content,orig_unit,cas_number
0,(capsicum anuum),Quercetin,0,5.070000,mg/100 g,OC1=CC2=C(C(O)=C1)C(=O)C(O)=C(O2)C1=CC=C(O)C(O...
1,(cucumis melo),Delphinidin,0,0.000000,mg/100 g,OC1=CC2=[O+]C(=C(O)C=C2C(O)=C1)C1=CC(O)=C(O)C(...
2,(ipomoea batatas),Luteolin,0,0.020000,mg/100 g,OC1=CC2=C(C(O)=C1)C(=O)C=C(O2)C1=CC=C(O)C(O)=C1
3,(lactuca sativa var. crispa),Delphinidin,0,0.000000,mg/100 g,OC1=CC2=[O+]C(=C(O)C=C2C(O)=C1)C1=CC(O)=C(O)C(...
4,(medicago sativa),Luteolin,0,0.000000,mg/100 g,OC1=CC2=C(C(O)=C1)C(=O)C=C(O2)C1=CC=C(O)C(O)=C1
...,...,...,...,...,...,...
4941,yogurt,Sugars,0,19000.000000,mg/100g,OC[C@H]1O[C@@](CO)(O[C@H]2O[C@H](CO)[C@@H](O)[...
4942,yogurt parfait,Sugars,0,11680.000000,mg/100g,OC[C@H]1O[C@@](CO)(O[C@H]2O[C@H](CO)[C@@H](O)[...
4943,zucchini,Epicatechin 3-gallate,0,0.000000,mg/100g,OC1=CC(O)=C2C[C@@H](OC(=O)C3=CC(O)=C(O)C(O)=C3...
4944,zucchini,Quercetin 3-rutinoside,0,1.326667,mg/100g,C[C@@H]1O[C@@H](OC[C@H]2O[C@@H](OC3=C(OC4=CC(O...


In [101]:
# compound_df = pd.read_csv(compound_fp)
# for i in compound_df:
#     display(i.head(2).T)
#     tmp = i.loc[i.description == '350602-26-5', 'cas_number'].values
#     display(tmp)
#     break

In [51]:
input_json = {
      "title": "Tylenol Extra Strength Caplets with 500 mg Acetaminophen, 100 ct",
      "description": "100-ct. of Tylenol Extra Strength Caplets with acetaminophen as pain reliever and fever reducerProvides temporary relief of minor aches and pains and helps reduce feverEach extra strength caplet contains 500 mg of acetaminophen, an effective pain relief ingredientRelieves common pain symptoms including headache, backache and menstrual crampsAlso works as a fever reducer and helps alleviate pain symptoms related to the common coldWhen used as directed, this pain reliever has an excellent safety profileThis oral pain reliever can be used by adults and children who are 12 years and olderFrom the #1 doctor-recommended brand of pain reliever, experience pain relief with TYLENOL"
}

In [109]:
import re
# DDI
drug_info_comb_fp = './data/Drug_info_combined.csv'
drug_info_comb = pd.read_csv(drug_info_comb_fp, index_col = 'Unnamed: 0')
compounds = drug_info_comb.Name.dropna()

def fuzzy_search(input_json, output_fp = './DDI_input.txt', pools = compounds):    
    description = input_json['description'].lower()
    out = []
    for compound in pools:
        pattern = compound.strip().lower()
        if re.search(pattern, description):
            out.append(compound)
    if output_fp:
        with open(output_fp, 'a') as fw:
            to_write = '\n'+'\n'.join(out)
            fw.write(to_write)

def ingest_input(input_json):
    fuzzy_search(input_json['current_drug'])
    for other_drug in input_json['other_drug']:
        fuzzy_search(other_drug['current_drug'])
        
# DFI